In [1]:
import json
import os
import s3fs
import numpy as np
import fireducks.pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import warnings
from dotenv import load_dotenv

load_dotenv()
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

print(f"Nombre de sujets {len(df['phrase_text'])}")
print(f"Nombre de sujets différents {df['phrase_text'].nunique()}")

df_sol_by_phrases = (
df.groupby("phrase_text", as_index = False)["sol"].count()
)

print(df_sol_by_phrases[df_sol_by_phrases["sol"] > 1])

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])

df["sol"].value_counts(dropna=False,
                      normalize=True)

df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)







df["clean_phrase_text"] = df["phrase_text"].apply(preprocess_text)
df = df[df["clean_phrase_text"] != ""]

X = df["clean_phrase_text"]
y = df["sol"]

pipeline = Pipeline(
    [
        ("feature_extraction", TfidfVectorizer()),
        ("classifier", LogisticRegression())
    ]
)

scores = cross_val_score(pipeline, X, y, cv=10, scoring="accuracy")

np.mean(scores)
np.std(scores)

ModuleNotFoundError: No module named 'fireducks'